# Support Vector Classification

In [1]:
from sklearn.datasets import  load_iris

In [2]:
iris = load_iris()
data =  iris.data
target = iris.target

In [3]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3)

In [9]:
from sklearn.svm import SVC

In [10]:
model = SVC()

In [13]:
model.fit(X_train, y_train)

SVC()

In [15]:
predictions = model.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

In [19]:
accuracy = accuracy_score(y_test, predictions)

In [21]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9555555555555556


# Hyperparameter Tuning using GridSearchCV

In [24]:
from sklearn.model_selection import GridSearchCV

In [26]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  # Only applicable for 'poly' kernel
}

In [28]:
grid_search = GridSearchCV(model, param_grid, cv=3)

In [30]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'degree': [2, 3, 4],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']})

In [32]:
grid_search.best_params_

{'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}

In [34]:
best_model = grid_search.best_estimator_

In [36]:
best_model.fit(X_train, y_train)

SVC(C=10, degree=2, kernel='poly')

In [38]:
predictions = best_model.predict(X_test)

In [40]:
accuracy = accuracy_score(y_test, predictions)

In [42]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9333333333333333


# Hyperparameter Tuning using Optuna

In [45]:
import optuna

In [47]:
# Define objective function for Optuna
def objective(trial):
    C = trial.suggest_loguniform('C', 0.01, 100)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    
    # If kernel is poly, also optimize the degree
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree, random_state=42)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    Accuracy = accuracy_score(y_test, y_pred)
    return Accuracy

In [49]:
# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-03-01 14:38:07,618] A new study created in memory with name: no-name-1e782436-3c3b-49d2-9e2b-7e2727294cbd
C:\Users\suren\AppData\Local\Temp\ipykernel_7064\855879956.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.01, 100)
[I 2025-03-01 14:38:07,626] Trial 0 finished with value: 0.9555555555555556 and parameters: {'C': 0.14253130443226592, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.9555555555555556.
C:\Users\suren\AppData\Local\Temp\ipykernel_7064\855879956.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.01, 100)
[I 2025-03-01 14:38:07,631] Trial 1 finished with

In [51]:
study.best_params

{'C': 0.5820990996937819, 'kernel': 'rbf', 'gamma': 'scale'}

In [53]:
study.best_value

0.9777777777777777

In [55]:
best_params = study.best_params

In [59]:
best_model1 = SVC(**best_params)

In [61]:
best_model1.fit(X_train, y_train)

SVC(C=0.5820990996937819)

In [63]:
predictions = best_model1.predict(X_test)

In [65]:
accuracy = accuracy_score(y_test, predictions)

In [67]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9777777777777777
